In [4]:
# Light Intensity Control Simulation (P/PI/PID Controller + Interactive Dashboard)
# The goal is to simulate a light control system using PID, PI, and P controllers with interactive widgets.

# Import necessary libraries
import matplotlib.pyplot as plt  # For plotting the graphs
import numpy as np  # For numerical operations
import ipywidgets as widgets  # For interactive widgets
from IPython.display import display, clear_output  # For displaying widgets and clearing outputs
import json  # For saving best parameters to a JSON file

# --- Interactive Controls ---
# Creating widgets for user interaction
controller_type = widgets.ToggleButtons(
    options=['P', 'PI', 'PID'],  # Available controllers to switch between
    value='PID',  # Default controller
    description='Mode:',  # Label for the widget
    button_style='info'  # Style of the button
)

kp_slider = widgets.FloatSlider(value=0.5, min=0, max=2.0, step=0.05, description='Kp')  # Kp slider for P/PI/PID
ki_slider = widgets.FloatSlider(value=0.1, min=0, max=1.0, step=0.01, description='Ki')  # Ki slider for PI/PID
kd_slider = widgets.FloatSlider(value=0.05, min=0, max=1.0, step=0.01, description='Kd')  # Kd slider for PID
set_point_slider = widgets.IntSlider(value=70, min=0, max=100, step=1, description='Target (%)')  # Desired light level (target)
save_button = widgets.Button(description='💾 Save Best Params', button_style='success')  # Save button for best parameters

# --- PID Simulation Function ---
# This function will simulate the light intensity control based on the chosen controller and parameters
def run_pid_simulation(kp, ki, kd, set_point, mode):
    light_level = 30  # Initial light level (could be any value between 0 and 100)
    integral = 0  # Integral part of PID
    prev_error = 0  # Previous error value to compute the derivative

    light_history = []  # List to store light levels over time
    error_history = []  # List to store error values over time
    control_signal_history = []  # List to store control signals over time

    # Simulating over 50 time steps
    for t in range(50):
        ambient_change = np.sin(t / 6.0) * 3 + np.random.normal(0, 1)  # Ambient light change (sine wave with noise)
        error = set_point - light_level  # Calculate error
        integral += error if mode in ['PI', 'PID'] else 0  # Only use integral for PI/PID
        derivative = error - prev_error if mode == 'PID' else 0  # Derivative only for PID

        control_signal = kp * error + ki * integral + kd * derivative  # Calculate the control signal
        light_level += control_signal + ambient_change  # Update light level with control signal and ambient change
        light_level = max(0, min(100, light_level))  # Keep the light level within bounds (0-100)
        prev_error = error  # Update the previous error value

        light_history.append(light_level)  # Store the current light level
        error_history.append(error)  # Store the current error
        control_signal_history.append(control_signal)  # Store the control signal

    # Clearing output for a clean display of new results
    clear_output(wait=True)

    # Plotting the results (light level, error, control signal over time)
    fig, axs = plt.subplots(3, 1, figsize=(10, 12))

    # Plot light intensity over time
    axs[0].plot(light_history, label='Light Intensity', color='blue')
    axs[0].axhline(set_point, color='red', linestyle='--', label='Target')
    axs[0].set_title('Light Intensity Over Time')
    axs[0].set_ylabel('Intensity (%)')
    axs[0].legend()
    axs[0].grid(True)

    # Plot error over time
    axs[1].plot(error_history, label='Error', color='orange')
    axs[1].set_title('Error Over Time')
    axs[1].set_ylabel('Error')
    axs[1].legend()
    axs[1].grid(True)

    # Plot control signal over time
    axs[2].plot(control_signal_history, label='Control Signal', color='green')
    axs[2].set_title('Control Signal Over Time')
    axs[2].set_ylabel('Signal Value')
    axs[2].legend()
    axs[2].grid(True)

    plt.tight_layout()  # Adjust layout for better appearance
    plt.show()  # Display all plots

# --- Save Best Parameters ---
# Function to save the best parameters (used in the "Save Best Params" button)
def save_params(btn):
    params = {
        'controller': controller_type.value,  # Selected controller type (P/PI/PID)
        'Kp': kp_slider.value,  # Kp value
        'Ki': ki_slider.value,  # Ki value
        'Kd': kd_slider.value,  # Kd value
        'Target': set_point_slider.value  # Target light level
    }
    with open("best_params.json", "w") as f:  # Save to a JSON file
        json.dump(params, f)
    print("Parameters saved to best_params.json")  # Notify the user

# Linking the save button with the function
save_button.on_click(save_params)

# --- Interactive Output ---
# Display the UI and the simulation outputs (the controller widgets and plots)
ui = widgets.VBox([controller_type, kp_slider, ki_slider, kd_slider, set_point_slider, save_button])  # Arrange widgets
out = widgets.interactive_output(run_pid_simulation, {  # Link function with inputs
    'kp': kp_slider,
    'ki': ki_slider,
    'kd': kd_slider,
    'set_point': set_point_slider,
    'mode': controller_type
})

display(ui, out)  # Display everything


Output()